## Yammer business background

Yammer is a social network for communicating with coworkers. Individuals share documents, updates, and ideas by posting them in groups. Yammer is free to use indefinitely, but companies must pay license fees if they want access to administrative controls, including integration with user management systems like ActiveDirectory.

Yammer has a centralized Analytics team, which sits in the Engineering organization. Their primary goal is to drive better product and business decisions using data. They do this partially by providing tools and education that make other teams within Yammer more effective at using data to make better decisions. They also perform ad-hoc analysis to support specific decisions.

## Table of Content

&#8195;<li>[**Libraries and Dependencies**](#-Libraries-and-Dependencies)

&#8195;<li>[**Datasets**](#Datasets)
     
&#8195;<li>[**Content**](#Content)

&#8195;&#8195;&#8195;[Part 1. Investigating a Drop in User Engagement](#Part-1.-Investigating-a-Drop-in-User-Engagement)

&#8195;&#8195;&#8195;[Part 2. Understanding Search Functionality](#Part-2.-Understanding-Search-Functionality)

&#8195;&#8195;&#8195;[Part 3. Validating A/B Test Results](#Part-3.-Validating-A/B-Test-Results)

&#8195;<li>[**Reference**](#Reference) 

## Libraries and Dependencies

In [361]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import datetime
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [381]:
init_notebook_mode(connected=True)

In [376]:
# import  plotly.offline  as offline
# offline.init_notebook_mode()

## Datasets
&#8195;[**Full explanations and overview of the following datasets: (NBviewer)**](http://nbviewer.jupyter.org/github/YuexiSC/business-and-data-analytics/blob/master/Projects/Yammer-Engagement-Analysis/Dataset_overview.ipynb)
1. Users
2. Events
3. Email Events
4. Rollup Periods

In [311]:
#Load Data
user=pd.read_csv('./dataset/yammer_users.csv')
events=pd.read_csv('./dataset/yammer_experiments.csv') 
email_events=pd.read_csv('./dataset/yammer_emails.csv') 
rollup_periods=pd.read_csv('./dataset/dimension_rollup_periods.csv') 

# Content

## Part 1. Investigating a Drop in User Engagement

#### 1. Visualize latest activity on the user engagement

In [312]:
events.head(1)

,user_id,occurred_at,event_type,event_name,location,device,user_type
0,10522.0,2014-05-02 11:02:39,engagement,login,Japan,dell inspiron notebook,3.0


In [313]:
event_daily_active=events[ (events.event_name =='login') & (events.event_type=='engagement')]
event_daily_active.head(1)

,user_id,occurred_at,event_type,event_name,location,device,user_type
0,10522.0,2014-05-02 11:02:39,engagement,login,Japan,dell inspiron notebook,3.0


In [318]:
event_daily_active['occurred_at'] = pd.to_datetime(event_daily_active['occurred_at']).dt.date

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [353]:
event_weekly_active=event_daily_active.set_index(pd.to_datetime(event_daily_active.occurred_at)).\
groupby(pd.Grouper(freq='W')).user_id.nunique()

In [416]:
iplot({'data': [{"x":event_weekly_active.index, "y": event_weekly_active}],
       'layout': {'title': 'Weekly Active Users',
                          'font': dict(size=16),       
                  'xaxis': {'title':'Date'},
                  'yaxis': {'title':'Number of Active users',
                           'titlefont': {'family':'default',
                                 'size':18,
                                  #'color':'black'
                                        }}
      } 
      })

In [189]:
#Transform 'Activated_at' to Datatime type
user['activated_at']= pd.to_datetime(user['activated_at']).dt.date
user['created_at']=pd.to_datetime(user.created_at).dt.date

In [190]:
#Ensure that the User_id is unique which can count the grouped activated day 
user[user.activated_at.notnull()].user_id.value_counts().sort_values(ascending=False).head()

0.0        1
15160.0    1
6.0        1
7.0        1
8.0        1
Name: user_id, dtype: int64

In [247]:
#Selected time period in from 2014-06-01 to 2014-09-01 (Given the hidden drop)
time_selected=user[(user.created_at >= datetime.date(2014,4,28)) & (user.created_at <= datetime.date(2014,9,1))]
time_selected.sample(3)

,user_id,created_at,company_id,language,activated_at,state
15511,15511.0,2014-07-08,11032.0,french,2014-07-08,active
18688,18688.0,2014-08-26,25.0,english,2014-08-26,active
17889,17889.0,2014-08-14,7.0,spanish,NaN,pending


In [340]:
#Group by activate date to get each day's engagement
engegament_all_users=time_selected.groupby(time_selected.created_at).count()
engegament_all_users.head(3)

,user_id,company_id,language,activated_at,state
created_at,,,,,
2014-04-28,65,65,65,39,65
2014-04-29,54,54,54,27,54
2014-04-30,63,63,63,25,63


In [341]:
#engegament_all_users=pd.to_datetime(engegament_all_user.index)

In [342]:
weekly_all_users = engegament_all_users.set_index(pd.to_datetime(engegament_all_users.index))
weekly_all_users.head()

,user_id,company_id,language,activated_at,state
created_at,,,,,
2014-04-28,65,65,65,39,65
2014-04-29,54,54,54,27,54
2014-04-30,63,63,63,25,63
2014-05-01,73,73,73,34,73
2014-05-02,57,57,57,30,57


In [339]:
weekly_all_users.user_id.resample('W').sum()

created_at
2014-05-04    353
2014-05-11    350
2014-05-18    362
2014-05-25    371
2014-06-01    366
2014-06-08    390
2014-06-15    413
2014-06-22    421
2014-06-29    404
2014-07-06    405
2014-07-13    424
2014-07-20    426
2014-07-27    458
2014-08-03    476
2014-08-10    406
2014-08-17    473
2014-08-24    468
2014-08-31    514
Freq: W-SUN, Name: user_id, dtype: int64

In [253]:
#Group by activate date to get each day's engagement
engegament_series=time_selected[time_selected.activated_at.notnull()].groupby(time_selected.activated_at).count()
engegament_series.head(3)

,user_id,created_at,company_id,language,activated_at,state
activated_at,,,,,,
2014-04-28,39,39,39,39,39,39
2014-04-29,27,27,27,27,27,27
2014-04-30,25,25,25,25,25,25


In [254]:
weekly_active_users = engegament_series.set_index(pd.to_datetime(engegament_series.index)).user_id
weekly_active_users.resample('W').sum()

activated_at
2014-05-04    172
2014-05-11    160
2014-05-18    186
2014-05-25    177
2014-06-01    186
2014-06-08    197
2014-06-15    198
2014-06-22    222
2014-06-29    210
2014-07-06    199
2014-07-13    223
2014-07-20    215
2014-07-27    228
2014-08-03    234
2014-08-10    189
2014-08-17    250
2014-08-24    259
2014-08-31    266
Freq: W-SUN, Name: user_id, dtype: int64

## Part 2. Understanding Search Functionality

## Part 3. Validating A/B Test Results

## Reference

1). [Working_with_time_series_data](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html)   
2). [Summarising_Aggregation_and_Grouping_in_Python](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)  
3). [SQL_Python1/2](https://blog.modeanalytics.com/group-by-sql-python/)  
&#8194;&#8194;&#8195;[SQL_Python2/2: Set_operation](https://blog.modeanalytics.com/set-operations-python-sql/)